In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


ModuleNotFoundError: No module named 'sklearn'

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


ModuleNotFoundError: No module named 'sklearn'

In [2]:
!pip install scikit-learn


   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.1 MB 2.1 MB/s eta 0:00:04
   ----- ---------------------------------- 1.0/8.1 MB 2.0 MB/s eta 0:00:04
   ------- -------------------------------- 1.6/8.1 MB 2.1 MB/s eta 0:00:04
   ---------- ----------------------------- 2.1/8.1 MB 2.2 MB/s eta 0:00:03
   ----------- ---------------------------- 2.4/8.1 MB 2.2 MB/s eta 0:00:03
   ------------ --------------------------- 2.6/8.1 MB 2.1 MB/s eta 0:00:03
   -------------- ------------------------- 2.9/8.1 MB 1.9 MB/s eta 0:00:03
   --------------- ------------------------ 3.1/8.1 MB 1.7 MB/s eta 0:00:03
   --------------- ------------------------ 3.1/8.1 MB 1.7 MB/s eta 0:00:03
   ---------------- ----------------------- 3.4/8.1 MB 1.6 MB/s eta 0:00:03
   ------------------ --------------------- 3.7/8.1 MB 1.5 MB/s eta 0:00:03
   ------------------- ---

In [ ]:
!pip install scikit-learn


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


In [4]:
df = pd.read_csv("master_dataset_final.csv")
print(df.shape)
df.head()


(2500, 4)


,release_date,title,main_director,soup
0,2015-06-17,Minions,kylebalda,assist aftercreditssting duringcreditssting ev...
1,2017-05-30,Wonder Woman,pattyjenkins,dccomic hero greekmytholog island worldwari su...
2,2017-03-16,Beauty and the Beast,billcondon,franc magic castl fairytal music curs anthropo...
3,2017-06-28,Baby Driver,edgarwright,robberi atlanta music crimeboss romanc tinnitu...
4,2014-10-24,Big Hero 6,chriswilliams,brotherbrotherrelationship hero talent reveng ...


In [5]:
count = CountVectorizer(
    analyzer='word',
    ngram_range=(1, 2),   # unigrams + bigrams
    min_df=3,             # ignore rare words
    stop_words='english'
)


In [6]:
count_matrix = count.fit_transform(df['soup'])

print(count_matrix.shape)


(2500, 3649)


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix)

print(cosine_sim.shape)


(2500, 2500)


In [8]:
import numpy as np

cosine_sim_df = pd.DataFrame(cosine_sim)
cosine_sim_df.to_csv("cosine_similarity_matrix.csv", index=False)

print("Cosine similarity matrix saved ✔️")


Cosine similarity matrix saved ✔️


In [9]:
import os
os.listdir()


['.ipynb_checkpoints',
 'bargraph.jpg',
 'cosine_similarity_matrix.csv',
 'credits.csv',
 'first.ipynb',
 'keywords.csv',
 'links.csv',
 'master_dataset.csv',
 'master_dataset_final.csv',
 'master_dataset_new.csv',
 'movie data.csv',
 'movies_metadata.csv',
 'week2_genre_analysis.ipynb',
 'Week3.1.ipynb',
 'week3.ipynb',
 'week3_data_processing_1.ipynb',
 'Week4.ipynb',
 'week4_data_processing_2.ipynb',
 'Week5.ipynb']

In [10]:
# reset index to be safe
df = df.reset_index(drop=True)

# create title → index mapping
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

indices.head()


title
Minions                 0
Wonder Woman            1
Beauty and the Beast    2
Baby Driver             3
Big Hero 6              4
dtype: int64

In [11]:
def recommend(movie_title, top_n=10):
    
    if movie_title not in indices:
        return f"Movie '{movie_title}' not found in dataset."

    idx = indices[movie_title]

    
    sim_scores = list(enumerate(cosine_sim[idx]))

    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    
    sim_scores = sim_scores[1:top_n+1]

   
    movie_indices = [i[0] for i in sim_scores]

    
    return df.loc[movie_indices, ['title', 'main_director', 'release_date']]
